<a href="https://colab.research.google.com/github/KimJisanER/medical_ai/blob/main/%5Bopen%5D_04_svpred_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 의료인공지능 전문가 양성과정 2022
## VitalDB Tutorial <br> Stroke volume prediction using arterial wave - preprocessing
- Date : Sep. 03, 2022
- Author : **Hyun-Lim Yang, Ph.D.**<br>
Research Assistant Professor @
Seoul National University Hospital <br>
Department of Anesthesiology and Pain Medicine
- E-mail : hlyang{_at_}snu{_dot_}ac{_dot_}kr
***

In [1]:
from IPython.display import HTML
style_warn = "<style>div.warn { background-color: #fcf2f2;border-color: #dFb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;}</style>"
HTML(style_warn)

### Import packages

<div class="warn">**Warning!** : use your directory at `download_directory`</div>


> **Wargning** <br>
> colab 환경을 위한 google drive import 코드가 포함되어 있습니다. <br>
> 로컬 환경에서 실행 시 colab을 위한 import function들을 comment out 한 뒤 실행하세요. 

In [2]:
!pip install vitaldb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive  # for colab
drive.mount('/content/gdrive/')  # for colab

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [4]:
import os
cloud_directory = '/content/gdrive/My Drive/KOHI_2022_CNN_data_open/'
os.listdir(cloud_directory) # for colab, check cloud directory mount

['datasets',
 'kohi_preprocessor.py',
 'sample_csv',
 '__init__.py',
 '__pycache__',
 'kohi_CNN_model_archi.png']

In [5]:
import sys
#download_directory = os.getcwd() # for local environments
#sys.path.append(download_directory) # for local environments
sys.path.append(cloud_directory) # for colab

In [6]:
import os
import sys
# download_directory = os.getcwd() # for local environments
# sys.path.append(download_directory) # for local environments

In [7]:
import numpy as np
import pandas as pd
import glob
import kohi_preprocessor as pre
import vitaldb
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')

download_directory = cloud_directory # for colab

### Data loading
샘플 파일을 vitaldb 서버로 부터 직접 다운받아 로드

> **TODO:** `00019.vital`을 100hz로 변환하여 `chart_pd_01`에 DataFrame으로 저장하기

In [8]:
track_names = ["EV1000/SV", "SNUADC/ART"]
### =========== Your code here ====================

vitalfile = vitaldb.VitalFile(19,track_names=track_names)
chart_pd_01 = vitalfile.to_pandas(track_names, interval=1/100)

### ===============================================

print(chart_pd_01.columns)

Index(['EV1000/SV', 'SNUADC/ART'], dtype='object')


In [9]:
col_svs = 'EV1000/SV'
col_art = 'SNUADC/ART'

### Stroke volume 데이터 추출

In [10]:
# stroke volume 데이터 추출
### =========== Your code here ====================

svs_data_pd = chart_pd_01[col_svs][chart_pd_01[col_svs].notnull()]
svs_index = svs_data_pd.index.values

### ===============================================
print(svs_data_pd.head())

53845    61.0
54008    61.0
54208    61.0
54408    61.0
54607    61.0
Name: EV1000/SV, dtype: float32


### arterial wave 데이터 추출 및 nan 지우기

In [11]:
# art 데이터 전체 추출 및 nan value 채우기
### =========== Your code here ====================

art_full_pd = chart_pd_01[col_art]
art_full_pd = art_full_pd.fillna(0)

### ===============================================

print(art_full_pd.head())

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: SNUADC/ART, dtype: float32


### 데이터셋 만들기
필요한 파라미터들 정의

In [12]:
# 필요한 파라미터들 정의
srate = 100
length = 20
max_limit_svs = 200 # svs max : 200
min_limit_svs = 20  # svs min : 20

입력 데이터 길이가 20s이니, svs가 20s 이내에 등장하는 것은 무시함

In [14]:
# svs index를 20초 뒤로 미룸
### =========== Your code here ====================

svs_points = svs_index[svs_index > length*srate]

### ===============================================

print(svs_points)

[  53845   54008   54208 ... 2700200 2700400 2700600]


### Arterial wave segment 추출

In [15]:
# arterial segment 추출하기
### =========== Your code here ====================
svs_values_list = [svs_data_pd[idx] for idx in svs_points]
art_seg_list = [art_full_pd[idx-(length*srate):idx].values for idx in svs_points]
### ===============================================

svs_values_np = np.array(svs_values_list)
art_seg_np = np.array(art_seg_list)

print(svs_values_np)
print(art_seg_np)

[61. 61. 61. ... 63. 63. 63.]
[[ 62.2872   62.2872   58.3374  ...  62.2872   65.2496   66.2371 ]
 [ 54.3876   56.3625   62.2872  ...  53.4001   51.4252   50.4377 ]
 [ 65.2496   64.2621   63.2747  ...  75.1242   70.1869   67.2245 ]
 ...
 [  6.00217 -13.747    30.6886  ...  -9.79714  -5.84731  19.8266 ]
 [-26.5839   23.7764   10.9395  ...   2.05234 -16.7093   28.7137 ]
 [-24.609    21.8015    4.02726 ...  -3.8724  -10.7846   20.814  ]]


### 조건에 따라 filter들 정의

In [16]:
# 필터들 선언
# svs min-max filter

### =========== Your code here ====================

svs_max_filter = svs_values_np < max_limit_svs
svs_min_filter = svs_values_np > min_limit_svs
svs_filter = svs_max_filter & svs_min_filter
### ===============================================


# abp range filter

### =========== Your code here ====================

art_filter_list = []
for seg in art_seg_np:
    filter_value = (np.array(seg) > 25.0).all() and (np.array(seg) < 250.0).all()
    art_filter_list.append(filter_value)
art_filter = np.array(art_filter_list)

# .all()로 모든 구간에서 만족하도록 한다

### ===============================================


# mstds 필터

### =========== Your code here ====================

mstds_values_list = []
for seg in tqdm(art_seg_np):
    if (np.array(seg) < 0.).any():
        mstds_values_list.append(float(0.))
    else:
        mstd_val, _ = pre.process_beat(seg)
        mstds_values_list.append(mstd_val)
mstds_filter = np.array(mstds_values_list) > 0.

### ===============================================


100%|██████████| 13217/13217 [03:43<00:00, 59.11it/s]


The all() function returns True if all items in an iterable are true, otherwise it returns False. If the iterable object is empty, the all() function also returns True.

전체 필터 하나로 만들기

In [17]:
### =========== Your code here ====================


all_filters = svs_filter & art_filter & mstds_filter

### ===============================================


### 필터 적용하여 데이터 추출

In [18]:
# 필터 적용해서 추출

### =========== Your code here ====================

svs_filtered = svs_values_np[all_filters]
art_filtered = art_seg_np[all_filters]

### ===============================================

print(svs_filtered.shape)
print(art_filtered.shape)

(12363,)
(12363, 2000)


데이터셋 정의

In [19]:
x_data = art_filtered
y_label = svs_filtered

In [20]:
print(x_data.shape)
print(y_label.shape)

(12363, 2000)
(12363,)
